In [1]:
import os
import json
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [2]:
dir = os.getcwd()
crop_params_path = os.path.join(dir, '../data/preprocessed/crop_parameters.csv')
crop_params_df = pd.read_csv(crop_params_path)
crop_params_df.head()

,group,crop,holos_crop_name,condition,moisture,R_p,R_s,R_r,R_e,N_p,N_s,N_r,N_e
0,Small-grain cereals,Small_grain_cereals,Small grain cereals,Rainfed,12,0.320,0.511,0.102,0.067,23.9,6.0,10.0,10.0
1,Small-grain cereals,Wheat,Wheat,<200,12,0.219,0.551,0.136,0.095,27.9,8.6,13.4,13.4
2,Small-grain cereals,Wheat,Wheat,200-350,12,0.244,0.518,0.147,0.091,26.3,8.2,10.4,10.4
3,Small-grain cereals,Wheat,Wheat,>350,12,0.431,0.488,0.049,0.032,56.2,6.5,11.9,11.9
4,Small-grain cereals,Barley,Barley,<350,12,0.451,0.400,0.090,0.059,19.0,7.0,10.0,10.0


In [3]:
def get_crop_params(crop, P, PE):
    # Filter the dataframe for the given crop
    crop_params = crop_params_df[crop_params_df['crop'] == crop]

    # If only one row matches, return its parameters
    if len(crop_params) == 1:
        return crop_params.iloc[0].to_dict()

    # Process each row to find the best match according to the rules
    for _, row in crop_params.iterrows():
        condition = row['condition']
        
        # Handle 'Canada'
        if condition == 'Canada':
            return row.iloc[0].to_dict()
        
        # Handle 'irrigated' and 'rainfed' based on P and PE
        if P < PE:
            if condition == 'irrigated':
                return row.to_dict()
        else:
            if condition == 'rainfed':
                return row.to_dict()
        
        # Handle numeric conditions with <, >, -
        if '<' in condition:
            upper_bound = float(condition.replace('<', ''))
            if PE - P < upper_bound:
                return row.to_dict()
                
        elif '>' in condition:
            lower_bound = float(condition.replace('>', ''))
            if PE - P > lower_bound:
                return row.to_dict()
                
        elif '-' in condition:
            lower_bound, upper_bound = map(float, condition.split('-'))
            if lower_bound <= PE - P <= upper_bound:
                return row.to_dict()

    # Default return if no specific conditions met
    return None

In [4]:
# Example usage
crop_name = 'Wheat'  # Example crop name
p = 300  # Example P value
pe = 200  # Example PE value
selected_params = get_crop_params(crop_name, p, pe)

In [5]:
selected_params

{'group': 'Small-grain cereals',
 'crop': 'Wheat',
 'holos_crop_name': 'Wheat',
 'condition': '<200',
 'moisture': 12,
 'R_p': 0.219,
 'R_s': 0.551,
 'R_r': 0.136,
 'R_e': 0.095,
 'N_p': 27.9,
 'N_s': 8.6,
 'N_r': 13.4,
 'N_e': 13.4}

In [7]:
# Example usage
crop_name = 'Wheat'  # Example crop name
p = 100  # Example P value
pe = 500  # Example PE value
selected_params = get_crop_params(crop_name, p, pe)
selected_params

{'group': 'Small-grain cereals',
 'crop': 'Wheat',
 'holos_crop_name': 'Wheat',
 'condition': '>350',
 'moisture': 12,
 'R_p': 0.431,
 'R_s': 0.488,
 'R_r': 0.049,
 'R_e': 0.032,
 'N_p': 56.2,
 'N_s': 6.5,
 'N_r': 11.9,
 'N_e': 11.9}